In [469]:
import pandas as pd
from collections import defaultdict


In [551]:
exit_price = 200
fin_independence = 15
redistribution_tax = .5

#Shareholders not subject to progressive equity, such as investors{name: % equity, }
std_equity_input = {'angel1': [.1, 0, 0] , 'venture1': [.1, 0, 0] }
std_equity_definitions = ['Equity Holder',
                          'Equity Percent (0)', 
                          'Standard Payout(1)', 
                          'Percent of Total Payout(2)']

#Shareholders that are subject to progressive equity{name: % equity, eligible for redistribution(boolean)}
prog_equity_input = {'founder1': [0.25, 0, False, True, 0, 0, 0, 0] , 
               'founder2' : [0.25, 0, False, True, 0, 0, 0, 0] , 
               'e1': [.1, 0, False, True, 0, 0, 0, 0], 
               'e2' : [.05, 0, False, True, 0, 0, 0, 0], 
               'e3' : [.05, 0, False, False, 0, 0, 0, 0], 
               'e4' : [.05, 0, False, False, 0, 0, 0, 0],
                'e5': [.05, 0, False, True, 0, 0, 0, 0]}

prog_equity_definitions = ['Equity Holder',
                           'Equity Percent(0)',  
                           'Standard Payout(1)',
                           'Triggered(2)',
                           'Eligible for Redistribution(3)',
                           'Progressive Payout Amount(4)',
                           'Paid(5)',
                           'Received(6)',
                           '% Total Payout(7)']

In [537]:
def check_equity_balance(std_equity, prog_equity):
    '''
    Checks to ensure that the equity values inputted add up to 1. 
    
    '''
    total_equity = 0
    for data in std_equity.values():
        equity = data[0]
        total_equity += equity
    for data in prog_equity.values():
        equity = data[0]
        total_equity += equity
        
    if total_equity != 1:
        print("Error in equity input. Total equity does not add up to 1. \n" 
              "Total equity = ", total_equity, '.')

In [538]:
def standard_payout(exit_price, std_equity, prog_equity):
    '''
    Calculate payout that each shareholder would receive under a standard equity scheme. 
    
    Parameters
    ----------
    
    Results
    -------
    
    
    ''' 
    for name, data in std_equity.items():
        equity_percent = data[0]
        payout = (equity_percent * exit_price)
        std_equity[name][1] = payout
    
    
    for name, data in prog_equity.items():
        equity_percent = data[0]
        payout = (equity_percent * exit_price)
        prog_equity[name][1] = payout
    
    return std_equity, prog_equity

In [539]:
def triggered_list(prog_equity, fin_independence, redistribution_tax):
    '''
    Check if progressive equity holder will pass financial independence number and
    be triggered for equity tax.
    
    '''
    
    #Keeps track of who has been triggered in this round of calculations
    
    triggered_current = defaultdict(list)
    
    for name, data in prog_equity.items():
        payout = data[1]
        redistribution_received = data[6]
        triggered = data[2]
        if (payout + redistribution_received) > fin_independence and triggered == False: 
            tax_paid = (payout + redistribution_received - fin_independence) * redistribution_tax
            triggered_current[name] = tax_paid
            prog_equity[name][2] = True
            prog_equity[name][5] = tax_paid
    
    return triggered_current, prog_equity

In [540]:
def contribute_pool(triggered_current):
    '''
    Equity holders that have been 'triggered' make a contribution to the redistribution pool.
    
    '''
    
    redistribution_pool = 0
    
    for values in triggered_current.values():
        redistribution_pool += values
    
    return redistribution_pool
        
        
    
    

In [541]:
def redistribution_denominator(prog_equity):
    '''
    Calculate denominator that will be used to determine the share of the redistribution pool
    received by each employee. 
     
    '''
    
    total_equity = 0
    for data in prog_equity.values():
        equity_percent = data[0]
        triggered = data[2]
        eligible = data[3]
        if triggered == False and eligible == True:
            total_equity += equity_percent
    return total_equity

In [542]:
def distribute_pool(prog_equity, redistribution_pool, redistribution_denominator):
    '''
    Distribute funds in pool to eligible employees based on their proportion of the equity
    eligible for redistribution. The equity eligible for redistribution is the "redistribution
    denominator".
    
    '''
    for name, data in prog_equity.items():
        equity_percent = data[0]
        std_payout = data[1]
        triggered = data[2]
        eligible = data[3]
        if triggered == False and eligible == True:
            redistribution_percent = equity_percent / redistribution_denominator
            redistribution_received = redistribution_percent * redistribution_pool
            prog_equity[name][6] = redistribution_received
        
    return prog_equity

In [543]:
def prog_payout(prog_equity):
    '''
    Subtract tax paid from and add redistributions received to standard equity payout 
    to calculate progressive equity payout.
    
    '''
    for name, data in prog_equity.items():
        standard_payout = data[1]
        tax_paid = data[5]
        redistribution_received = data[6]
        prog_equity[name][4] = standard_payout - tax_paid + redistribution_received
    return prog_equity

In [544]:
def percent_final_payout(prog_equity, std_equity, exit_price):
    '''
    Calculate the percent of the total exit price received by each progressive equity holder.
    
    '''
    for name, data in prog_equity.items():
        prog_equity[name][7] = data[4] / exit_price
    
    for name, data in std_equity.items():
        std_equity[name][2] = data[1] / exit_price
    
    return prog_equity, std_equity
    

In [545]:
def test(std_equity, prog_equity, exit_price):
    '''
    Test to check that total payout equals the exit price. 
    
    '''
    test_exit_price = 0
    for values in std_equity.values():
        std_payout = values[1]
        test_exit_price += std_payout
    for values in prog_equity.values():
        prog_payout = values[4]
        test_exit_price += prog_payout
    
    if test_exit_price == exit_price:
        print("Total Payouts: ", test_exit_price)
        print("The total payouts equal the exit price.")

    else:
        print("Total Payouts: ", test_exit_price)
        print("The total payouts do not equal the exit price.\n \
        There is an error or excess in the redistribution pool.")


In [546]:
def check_payout_percents(std_equity, prog_equity):
    total_payout_percents = 0
    for data in std_equity.values():
        payout_percent = data[2]
        total_payout_percents += payout_percent
    for data in prog_equity.values():
        payout_percent = data[5]
        total_payout_percents += payout_percent
        
    if total_payout_percents != 1:
        print("Error in payout percent calculation. Total does not add up to 1. \n" 
              "Total Payout Percent = ", total_payout_percents, '.')
    

In [552]:
def calculate_payouts(exit_price, fin_independence, redistribution_tax, 
                      std_equity, std_equity_definitions,
                      prog_equity, prog_equity_definitions):
    '''
    Calculate payouts for each equity holder in a progressive equity scheme. 
    
    '''
    #Check equity inputted for each equity holder totals to 1.
    check_equity_balance(std_equity, prog_equity)
    
    #Calculate the standard payout
    std_equity, prog_equity = standard_payout(exit_price, std_equity, prog_equity)
    
    #Determine who would be triggered for the equity tax
    triggered_current, prog_equity = triggered_list(prog_equity, fin_independence, 
                                                    redistribution_tax)

    #Contribute funds to the redistrubtion pool
    redistribution_pool = contribute_pool(triggered_current)

    #Calculate the denominator used to determine the distribution of the pool
    denominator = redistribution_denominator(prog_equity)

    #Distribute the pool
    prog_equity = distribute_pool(prog_equity, redistribution_pool, denominator)
    
    #Check if any of the equity holders that received distribution should be triggered 
    #for the tax. 
    triggered_current, prog_equity = triggered_list(prog_equity, fin_independence, 
                                                    redistribution_tax)
    
    #Contribute funds to the redistribution pool
    redistribution_pool = contribute_pool(triggered_current)
    
    #Calculate progressive equity payouts
    prog_equity = prog_payout(prog_equity)
    
    #Calculate the percent of the total exit price received by each progressive equity holder
    prog_equity, std_equity = percent_final_payout(prog_equity, std_equity, exit_price)
    
    #Check that total payouts equal the exit price
    test_result = test(std_equity, prog_equity, exit_price)
    
    print("Exit Price: ", exit_price, '\n')
    print("Financial Independence :", fin_independence, '\n')
    print("Equity Tax: ", redistribution_tax, '\n')
    print("Excess in Redistribution Pool: ", redistribution_pool, '\n')
    
    df_prog_equity = pd.DataFrame.from_dict(prog_equity, 'index')
    df_prog_equity.reset_index(inplace=True)
    df_prog_equity.columns = prog_equity_definitions
    
    df_std_equity = pd.DataFrame.from_dict(std_equity, 'index')
    df_std_equity.reset_index(inplace=True)
    df_std_equity.columns = std_equity_definitions
    

    return df_std_equity, df_prog_equity
        

In [553]:
df_std_equity, df_prog_equity = calculate_payouts(exit_price, fin_independence, redistribution_tax, 
                                            std_equity_input, std_equity_definitions,
                                            prog_equity_input, prog_equity_definitions)


Total Payouts:  186.25
The total payouts do not equal the exit price.
         There is an error or excess in the redistribution pool.
Exit Price:  200 

Financial Independence : 15 

Equity Tax:  0.5 

Excess in Redistribution Pool:  13.75 



In [554]:
df_std_equity

,Equity Holder,Equity Percent (0),Standard Payout(1),Percent of Total Payout(2)
0,angel1,0.1,20.0,0.1
1,venture1,0.1,20.0,0.1


In [555]:
df_prog_equity

,Equity Holder,Equity Percent(0),Standard Payout(1),Triggered(2),Eligible for Redistribution(3),Progressive Payout Amount(4),Paid(5),Received(6),% Total Payout(7)
0,founder1,0.25,50.0,True,True,32.500,17.500,0.00,0.162500
1,founder2,0.25,50.0,True,True,32.500,17.500,0.00,0.162500
2,e1,0.10,20.0,True,True,17.500,2.500,0.00,0.087500
3,e2,0.05,10.0,True,True,21.875,6.875,18.75,0.109375
4,e3,0.05,10.0,False,False,10.000,0.000,0.00,0.050000
5,e4,0.05,10.0,False,False,10.000,0.000,0.00,0.050000
6,e5,0.05,10.0,True,True,21.875,6.875,18.75,0.109375
